<a href="https://colab.research.google.com/github/rlagosb/taller_eiv/blob/main/3_Ejercicio_Cubo_multidimensional_(R).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio: creación de un cubo
En este ejercicio vamos a armar una tabla de datos multidimensional para el análisis de las brechas de consultas de especialidad médica (CNE). Para ello haremos lo siguiente:

1. Generar un modelo estrella con las métricas y dimensiones de análisis

2. Generar tablas fact y dimensiones

3. Exportar a excel

 # Configurar área de trabajo

In [ ]:
#borrar objetos previos
rm(list=ls())
#borrar figuras (si hay)
while(!dev.cur())dev.off()
cat("\014")

# Para manipulación de bases de datos. Si no existe, la instalamos
if(!require(dplyr)){install.packages("dplyr")}
# Para importar/exportar bases de datos. Si no existe, la instalamos
if(!require(rio)){install.packages("rio")}
# Para exportar archivos en formato parquet
if(!require(arrow)){install.packages("arrow")}
if(!require(nanoparquet)){install.packages("nanoparquet")}

# definir carpetas de trabajo
datos_normalizados <- 'https://github.com/rlagosb/taller_eiv/raw/refs/heads/main/datos_normalizados/'
datos_cubos <- './content/cubos/'

if (!dir.exists(datos_cubos)){dir.create(datos_cubos, recursive = TRUE)}

# cargar datos (concatenamos con paste0 la ubicación de github con el nombre de cada archivo *.parquet a trabajar)
centros <- rio::import(paste0(datos_normalizados, "Centros.parquet"))
especialidades <- rio::import(paste0(datos_normalizados, "Especialidades.parquet"))
demanda <- rio::import(paste0(datos_normalizados, "Demanda.parquet"))
produccion <- rio::import(paste0(datos_normalizados, "Produccion.parquet"))

# Crear tabla FACT
Construiremos una tabla con los campos:

- Centro_cod: código del centro de salud
- Especialidad_cod: código de la especialidad médica (prestación)
- Mes: mes del año 2021
- CNE_solicitadas: solicitudes de interconsultas recibidas por establecimiento, especialidad y mes
- CNE_producidas: consultas nuevas de especialidad producidas por establecimientos, especialidad y mes



In [ ]:
# Examinemos los campos que tienen las tablas de demanda y producción
# Con el cruce de estas tablas podemos obtener las métricas fundamentales para el análisis
#Tres primeros valores de la base demanda
demanda %>% head(3)

cat("\n\n") #saltos de línea

#Tres primeros valores de la base producción
produccion %>% head(3)

In [ ]:
# 1. Seleccionar y renombrar columnas de la tabla demanda
dda <- demanda %>%
  dplyr::select(Centro_destino, Especialidad, Mes, Interconsultas) %>%  # Seleccionar columnas relevantes
  dplyr::rename(
    Centro_cod = Centro_destino,       # Renombrar 'Centro_destino' a 'Centro_cod'
    Especialidad_cod = Especialidad,   # Renombrar 'Especialidad' a 'Especialidad_cod'
    CNE_solicitadas = Interconsultas    # Renombrar 'Interconsultas' a 'CNE_solicitadas'
  )

# 2. Seleccionar y renombrar columnas de la tabla produccion
prod <- produccion %>%
  dplyr::select(Centro, Codigo, Mes, Nuevas) %>%  # Seleccionar columnas relevantes
  dplyr::rename(
    Centro_cod = Centro,              # Renombrar 'Centro' a 'Centro_cod'
    Especialidad_cod = Codigo,        # Renombrar 'Codigo' a 'Especialidad_cod'
    CNE_producidas = Nuevas            # Renombrar 'Nuevas' a 'CNE_producidas'
  )

# 3. Realizar una unión completa (outer join) entre demanda y produccion
fact <- dda %>%
  dplyr::full_join(prod, by = c("Centro_cod", "Especialidad_cod", "Mes"))  # Unir por las columnas especificadas

# 4. Reemplazar valores NA por 0 en las columnas 'CNE_solicitadas' y 'CNE_producidas'
# Utilizamos la función 'coalesce' para reemplazar NAs con 0
fact <- fact %>%
  dplyr::mutate(
    CNE_solicitadas = coalesce(CNE_solicitadas, 0),   # Reemplazar NA en 'CNE_solicitadas' por 0
    CNE_producidas = coalesce(CNE_producidas, 0)    # Reemplazar NA en 'CNE_producidas' por 0
  )

# 5. Convertir las columnas 'CNE_solicitadas' y 'CNE_producidas' a enteros
fact <- fact %>%
  dplyr::mutate(
    CNE_solicitadas = as.integer(CNE_solicitadas),   # Convertir a entero
    CNE_producidas = as.integer(CNE_producidas)      # Convertir a entero
  )

# Mostrar la estructura de la tabla resultante
str(fact)

## 🏁 Discusión
1. ¿Cuáles son las métricas que registra esta tabla?
1. ¿Cuáles son los hechos que registra esta tabla? ¿Cuántos exiten?
2. ¿Cuál es el nivel de detalle con que se describen los hechos (granularidad)?
1. ¿A partir de esta tabla es posible consultar los hechos a nivel diario? ¿Trimestral? ¿Anual?

# Dimensión Periodo
Aunque nuestro análisis sólo considera un año, podríamos querer analizar los datos a nivel trimestral o anual. Construiremos una tabla con los campos

- Mes: llave primaria
- Trimestre
- Año

**⚠** Si trabajáramos con múltiples años necesitaríamos definir otra llave primaria para esta dimensión

In [ ]:
# Creamos un dataframe vacío para agregar los campos
dim_periodo <- cbind.data.frame(
  Mes =         1:12,               # Números del 1 al 12 para los meses
  Año =         2021,               # Año constante 2021
  Trimestre =   rep(1:4, each = 3)  # Asignar trimestre
) %>%
  dplyr::arrange(Mes)  # Ordenar por Mes para mantener el orden cronológico

dim_periodo

# Dimensión Especialidad
Crearemos una tabla con los siguientes campos:

- Especialidad_cod: código de la especialidad (llave primaria)
- Especialidad: nombre de la especialidad
- Alto_riesgo: especialidad con mayor mortalidad para tiempos de espera extensos según Martínez et al ([2019](https://doi.org/10.1186/s12889-019-6526-6))
- REM: si se incluye en reportes estadísticos mensuales


In [ ]:
# Examinemos la tabla especialidades para identificar los campos existentes y los que faltan
set.seed(123)#para reproducibilidad
especialidades[sample(nrow(especialidades),5),] #tomamos 5 observaciones al azar

In [ ]:
# Generemos la dimensión especialidad con una función especial

# 1. Seleccionar y renombrar columnas de la tabla especialidades
esp <- especialidades %>%
  dplyr::select(Codigo, Nombre, Codigo_rem) %>%  # Seleccionar columnas relevantes
  dplyr::rename(
    Especialidad_cod = Codigo,        # Renombrar 'Codigo' a 'Especialidad_cod'
    Especialidad = Nombre             # Renombrar 'Nombre' a 'Especialidad'
  )

# 2. Generar el campo Especialidad_REM
esp <- esp %>%
  dplyr::mutate(
    Especialidad_REM = ifelse(!is.na(Codigo_rem), "Si", "No")  # Asignar 'Si' si Codigo_rem no es NA, de lo contrario 'No'
  ) %>%
  dplyr::select(-Codigo_rem)  # Eliminar la columna 'Codigo_rem'

 # Identificamos las especialidades de alto riesgo: CARDIOLOGIA, CIRUGIA CARDIOVASCULAR, GASTROENTEROLOGIA ADULTO, NEFROLOGIA ADULTO, NEUROLOGIA ADULTO, UROLOGIA
  alto_riesgo <- c('07-003', '07-006', '07-018', '07-037', '07-041', '07-054')  # Lista de códigos de alto riesgo

  dim_especialidad <- esp %>%
    dplyr::mutate(
      Riesgo_mortalidad = ifelse(Especialidad_cod %in% alto_riesgo, "Si", "No")  # Asignar 'Si' si el código está en alto_riesgo, de lo contrario 'No'
    )

# Veamos el resultado
dplyr::glimpse(dim_especialidad)
set.seed(123)#para reproducibilidad
especialidades[sample(nrow(especialidades),5),] #tomamos 5 observaciones al azar

## 🏁 Discusión
1. ¿Qué diferencia a esta tabla de la tabla normalizada *Especialidades*?
2. ¿Cuál es el propósito de cada una?

# 🍎 Desafío: Crear Dimensión Establecimiento
Cree un dataframe `dim_centro` con los siguientes campos:

- Centro_cod: código DEIS del centro (llave primaria)
- Centro: nombre del centro de salud
- Complejidad: nivel de complejidad del centro
- Servicio: Servicio de Salud del cual depende

In [ ]:
dim_centro <- centros %>%
  dplyr::select(Codigo, Nombre, Dependencia, `Nivel de Complejidad`) %>%  # Seleccionar columnas relevantes. Ojo que para columnas que contienen espacios o parten con caracteres especiales, dplyr requirere el tilde inverso para llamarlos y manipularlos
  dplyr::rename(
    Centro_cod    = Codigo,
    Centro        = Nombre,
    Complejidad   = "Nivel de Complejidad",
    Servicio      = Dependencia
  )
#Miramos el resultado
dim_centro

# Cubo
Generamos una tabla multidimensional uniendo la tabla Fact con las dimensiones utilizando las llaves.

In [ ]:
cubo <- fact %>%
  dplyr::left_join(dim_especialidad, by = "Especialidad_cod") %>%  # Unir con dim_especialidad en 'Especialidad_cod'
  dplyr::left_join(dim_centro, by = "Centro_cod") %>%              # Unir con dim_centro en 'Centro_cod'
  dplyr::left_join(dim_periodo, by = "Mes")                        # Unir con dim_periodo en 'Mes'

set.seed(2125)
cubo %>%
  sample_n(5, replace = TRUE)

In [ ]:
# Exportamos a la carpeta con los cubos
 cubo%>%
  rio::export(paste0(datos_cubos, 'Cubo_brechas_CNE.xlsx'), rowNames = FALSE)

## 🏁 Discusión
1. ¿Qué campos tienen redundancias? (¿Cuál forma normal no se cumple?)
3. ¿Cuál es el propósito de introducir esta redundancia?
4. ¿Existe el riesgo que se produzan inconsistencias en los nombres de los centros o especialidades?
1. ¿Qué cambios tendría que hacer a esta tabla si tuviera:
  - Datos del 2021 al 2024?
  - Datos de más establecimientos?